In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution

warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
!pwd

/Users/kenobi/Documents/naarni/repo/explore_data/5.CanDataAnalysis/ver3


In [3]:
EXTRACTION_DIR = Path("../../../data_points/extracted_parts/261125_2days") 
# CORE_COLS = ["id", ]

In [4]:
def free_mem():
    """Try to return freed memory back to the OS (no-op on some platforms)."""
    try:
        libc = ctypes.CDLL(None)
        if hasattr(libc, "malloc_trim"):
            libc.malloc_trim(0)
    except Exception:
        pass

In [5]:
def drop_msb_and_convert(numbers_list):
    translated_numbers = []
    for number in numbers_list:
        binary_str = format(number, 'b')
        dropped_msb_str = binary_str[1:]
        if not dropped_msb_str:
            new_int_value = 0
        else:
            new_int_value = int(dropped_msb_str, 2)
        translated_numbers.append(new_int_value)
    return translated_numbers


In [6]:
import re

dbc_lines = open("canId_conversions.txt").read().splitlines()

temp_map = {}   # dbc_id → list of 1–108
volt_map = {}   # dbc_id → list of 1–576

current_dbc_id = None
current_type = None  # "temp" or "volt"

temp_pattern = re.compile(r"SG_\s+Pack_Temperature(\d+)")
volt_pattern = re.compile(r"SG_\s+Pack_cellVoltage_(\d+)")

for line in dbc_lines:
    line = line.strip()

    # Detect BO_ lines
    if line.startswith("BO_ "):
        parts = line.split()
        current_dbc_id = int(parts[1])
        name = parts[2]

        if name.startswith("Pack_Temperature"):
            current_type = "temp"
            temp_map[current_dbc_id] = []

        elif name.startswith("Pack_cellVoltage"):
            current_type = "volt"
            volt_map[current_dbc_id] = []

        else:
            current_type = None  # ignore unrelated frames

        continue

    # Inside a BO_ block → extract signal names
    if current_type == "temp":
        m = temp_pattern.search(line)
        if m:
            idx = int(m.group(1))
            temp_map[current_dbc_id].append(idx)

    elif current_type == "volt":
        m = volt_pattern.search(line)
        if m:
            idx = int(m.group(1))
            volt_map[current_dbc_id].append(idx)


In [7]:
print("Temperature blocks:", len(temp_map))
print("Total temperature indices:", sum(len(v) for v in temp_map.values()))

print("Voltage blocks:", len(volt_map))
print("Total voltage indices:", sum(len(v) for v in volt_map.values()))


Temperature blocks: 15
Total temperature indices: 108
Voltage blocks: 145
Total voltage indices: 576


In [8]:
# --- DUCKDB CHUNK GENERATOR (Fixed) ---

def duckdb_chunk_generator(conn, sql_query, chunk_size):
    """Generates Pandas DataFrames in chunks directly from DuckDB cursor."""
    cursor = conn.cursor() 
    cursor.execute(sql_query)
    
    while True:
        # Uses the corrected method name: fetch_df_chunk
        chunk = cursor.fetch_df_chunk(chunk_size) 
        if chunk is None or chunk.empty:
            break
        yield chunk

# --- ROBUST FILE EXTRACTION (Fixed from OSErrors) ---

def extract_files_to_disk(zip_path, output_dir):
    """Cleans directory and extracts all Parquet files from ZIP."""
    if output_dir.exists():
        logging.info(f"🧹 Clearing existing directory: {output_dir.resolve()}")
        # Robust cleanup to avoid OS/lock issues
        try:
            shutil.rmtree(output_dir)
        except OSError:
             for item in output_dir.iterdir():
                if item.is_dir():
                    shutil.rmtree(item)
                else:
                    os.remove(item) 
             os.rmdir(output_dir)

    output_dir.mkdir(parents=True)
        
    logging.info("🔄 Extracting ALL Parquet files from ZIP to disk...")
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            all_files_to_extract = [f for f in z.namelist() if f.endswith(".parquet")]
            logging.info(f"🔎 Found {len(all_files_to_extract)} total Parquet files in archive.")
            for filename in all_files_to_extract:
                z.extract(filename, path=output_dir)
            return len(all_files_to_extract)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ ZIP file not found at: {zip_path}") from None

def setup_duckdb_query(
    output_dir,
    utc_start,
    utc_end,
    core_cols=None,
    ids=None,   # NEW
):
    """
    Sets up DuckDB connection and SQL query for reading Parquet files
    with timestamp predicate-pushdown filtering.

    Behavior:
      - If core_cols is None -> SELECT * (all columns)
      - If core_cols is provided -> SELECT only those columns
      - If ids (list of strings) is provided -> adds id IN (...) filter
    """

    parquet_glob_path = str(output_dir.joinpath("**/*.parquet"))

    # SELECT clause
    if core_cols is None:
        select_clause = "SELECT *"
    else:
        col_list = ", ".join([f'"{c}"' for c in core_cols])
        select_clause = f"SELECT {col_list}"

    # ID filter clause
    id_clause = ""
    if ids:
        # Format like: '6','16','28'
        id_list = ", ".join([f"'{x}'" for x in ids])
        id_clause = f"AND id IN ({id_list})"

    # Full SQL
    sql_query = f"""
        {select_clause}
        FROM read_parquet('{parquet_glob_path}')
        WHERE 
            "timestamp" >= '{utc_start.isoformat()}' AND
            "timestamp" <  '{utc_end.isoformat()}'
            {id_clause}
    """

    return duckdb.connect(), sql_query


In [9]:
ids = ['28']


ist_start_str = "2025-11-27 13:00:00.000"
ist_end_str = "2025-11-27 13:10:00.000"

# Parse IST timestamp strings
ist_start = datetime.strptime(ist_start_str, "%Y-%m-%d %H:%M:%S.%f")
ist_end   = datetime.strptime(ist_end_str,   "%Y-%m-%d %H:%M:%S.%f")

# Convert IST -> UTC
utc_start = ist_start - timedelta(hours=5, minutes=30)
utc_end   = ist_end   - timedelta(hours=5, minutes=30)


# EXTRACTION_DIR should be a pathlib.Path
conn, sql_query = setup_duckdb_query(
    output_dir=EXTRACTION_DIR,
    utc_start=utc_start,
    utc_end=utc_end,
    core_cols=None,   # <-- ALL COLUMNS
    ids=ids,          # <-- Filter by IDs
)

df_raw = conn.execute(sql_query).fetchdf()
# Compute the correct position
ts_index = df_raw.columns.get_loc("timestamp")

# Insert at index+1
df_raw.insert(loc=ts_index + 1, column="timestamp_ist",
          value=(df_raw["timestamp"] + pd.Timedelta(hours=5, minutes=30))
                 .dt.strftime("%Y-%m-%d %H:%M:%S.%f"))        
df_raw["timestamp_ist"] = pd.to_datetime(df_raw["timestamp_ist"], format="%Y-%m-%d %H:%M:%S.%f")

In [10]:
df_raw.head().to_csv("df_raw_head.csv", index=False)

In [11]:
(df_raw.isna().sum()*100/len(df_raw)).round(2).to_csv("df_raw_na_counts.csv", index=True)

In [12]:
df_raw[df_raw.pack_temperature81.notna()].head()
# df_raw.head()

,id,timestamp,timestamp_ist,insert_timestamp,sequence,number_of_can_ids,number_of_records,percentage_of_can_ids,lowpressureoilpumpfaultcode,lowvoltagedcaclifesignal,batterycoolingstate,batterycoolanttemperature,odometerreading,brakepedalpos,ebsredwarningsignal,abs_ebsamberwarningsignal,insulation_status,insulation__value,lifesignalofinsulation,output_phase_voltage,output_phase_currant,motor_rpm,motor_torque,motor_temperature,igbt_temperature,mcuself_teststatus,side_door_panel_open_alarm,ignitionstatus,brakingsystemmallfunction_alarm,parking_brake_status,front_air_pressure,rear_air_pressure,brake_pad_worn_out_alarm,guna_dcp_temperature,guna_dcm_temperature,gunb_dcp_temperature,gunb_dcm_temperature,gunc_dcp_temperature,gunc_dcm_temperature,batterypackovervoltagealarm,monomerovervoltagealarm,polehightemperaturealarm,batteryhightemperaturealarm,hightempalarmofcharginggun,monomerundervoltagealarm,batterypackundervoltagealarm,temperaturedifferencealarm,singlevoltagedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,sochighalarm,soclowalarm,bms_fault_code,gun_connection_status,maxavailableshorttermcharging,maxavailableshorttimedischarge,chargingcontactor2positive,chargingcontactor2negative,chargingcontactor1positive,batterytotalnegativecontactor,chargingcontactor1negative,pack_negative_contactors_status,bat_soc,bat_voltage,total_battery_current,bmslifesignal,steering_pump_enable_command,air_compressor_enable_command,dcdc_enable_command,vcuversioninformation,distancetoempty,accelarationpedal,brake_pedal,vcuself_teststatus,systeminterlockstate,vehiclereadycondition,vehicle_speed_vcu,gear_position,autoholdunenable,powertrain_failure_alarm,vcu_fault_code,vehiclecontrollerlife,cellmax_voltagecellnumber,cell_max_voltage,cellminvoltagecellnumber,cell_min_voltage,pack1_cellmax_temperature,pack1_maxtemperature_cell_number,pack1_cell_min_temperature,pack1_celltemperature_cellnumber,fourinonelifesignal,fourinoneself_teststatus,prechargedcontactorstate,mainmotorcontactor1state,accessorycontactorstatus,pre_charge_status,fiveinone_faultcode,airpumpcondition,air_compreesor_temperature,airpumpdcaclifesignal,dcdc_statusandfailure,dcdc_out_put_currant,dcdc_out_put_volatge,dcdcbusvoltage,dcdclifesignal,oilpumpcondition,steering_pump_rpm,steeringcontroller_failurealarm,lift_lower_normal_request,kneeling_request,th_management_sys_working_mode,soh,charging_contactor_3_positive,charging_contactor_3_negative,charger_voltage,charger_current,chrargecurrent_request,evcc_error,chargingerror_bms,vehicle_operation_mode,pack_temperature1,pack_temperature2,pack_temperature3,pack_temperature4,pack_temperature5,pack_temperature6,pack_temperature7,pack_temperature8,pack_temperature9,pack_temperature10,pack_temperature11,pack_temperature12,pack_temperature13,pack_temperature14,pack_temperature15,pack_temperature16,pack_temperature17,pack_temperature18,pack_temperature19,pack_temperature20,pack_temperature21,pack_temperature22,pack_temperature23,pack_temperature24,pack_temperature25,pack_temperature26,pack_temperature27,pack_temperature28,pack_temperature29,pack_temperature30,pack_temperature31,pack_temperature32,pack_temperature33,pack_temperature34,pack_temperature35,pack_temperature36,pack_temperature37,pack_temperature38,pack_temperature39,pack_temperature40,pack_temperature41,pack_temperature42,pack_temperature43,pack_temperature44,pack_temperature45,pack_temperature46,pack_temperature47,pack_temperature48,pack_temperature49,pack_temperature50,pack_temperature51,pack_temperature52,pack_temperature53,pack_temperature54,pack_temperature55,pack_temperature56,pack_temperature57,pack_temperature58,pack_temperature59,pack_temperature60,pack_temperature61,pack_temperature62,pack_temperature63,pack_temperature64,pack_temperature65,pack_temperature66,pack_temperature67,pack_temperature68,pack_temperature69,pack_temperature70,pack_temperature71,pack_temperature72,pack_temperature73,pack_temperature74,pack_temperature75,pack_temperature76,pack_temperature77,pack_tempe

In [13]:
volt_cols = [c for c in df_raw.columns if c.startswith("pack_cellvoltage_")]
len(volt_cols)

576

In [17]:
# df_raw.isnull().sum()[(df_raw.isnull().sum()*100/len(df_raw))>98]*100/len(df_raw)
# Calculate the percentage of nulls for all columns
null_percentages = df_raw.isnull().sum() * 100 / len(df_raw)

# Filter the Series to only include columns with > 98% nulls
highly_null_cols_series = null_percentages[null_percentages > 98]

# Then proceed with conversion and saving as shown above.
highly_null_cols_series.to_excel("can_highly_null_columns.xlsx", index=True)

In [15]:
wait

NameError: name 'wait' is not defined

In [ ]:
def extract_voltage_sweeps(df, volt_cols, timestamp_col="timestamp_ist"):
    """
    Detects complete voltage sweeps (ignoring cells 81–84).
    Returns: cycle_start, cycle_end, duration_sec, interval_sec,
             + voltage values for all valid 572 cells.
    """

    # -------- STEP 1 — Identify valid columns (exclude 81–84) --------
    def get_cell_num(col):
        return int(col.split("_")[-1])

    ignore_cells = {81, 82, 83, 84}

    valid_volt_cols = [c for c in volt_cols if get_cell_num(c) not in ignore_cells]
    valid_cell_indices = [get_cell_num(c) for c in valid_volt_cols]

    EXPECTED = len(valid_cell_indices)  # should be 572
    print(f"Using {EXPECTED} expected cells (ignoring 81–84).")

    # -------- STEP 2 — Sort & prepare dataframe --------
    df = df.sort_values(timestamp_col).reset_index(drop=True)

    # For each row, identify which cells appear
    df["cells_present"] = df[valid_volt_cols].notna().apply(
        lambda row: [valid_cell_indices[i] for i, ok in enumerate(row) if ok],
        axis=1
    )

    # -------- STEP 3 — Sweep detection logic --------
    sweeps = []
    seen = set()
    cycle_start = None

    for _, row in df.iterrows():
        ts = row[timestamp_col]
        cells = row["cells_present"]

        if len(seen) == 0:
            cycle_start = ts

        seen.update(cells)

        # Full sweep detected (572 cells)
        if len(seen) == EXPECTED:
            sweeps.append({"cycle_start": cycle_start, "cycle_end": ts})
            seen = set()

    # -------- STEP 4 — No sweeps found --------
    if len(sweeps) == 0:
        print("⚠ No complete sweeps detected for 572 valid cells.")
        return pd.DataFrame(columns=["cycle_start","cycle_end","duration_sec","interval_sec"] + valid_volt_cols)

    # -------- STEP 5 — Build sweep dataframe --------
    df_cycles = pd.DataFrame(sweeps)
    df_cycles["duration_sec"] = (df_cycles["cycle_end"] - df_cycles["cycle_start"]).dt.total_seconds()
    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None

    # -------- STEP 6 — Extract voltage values for each sweep --------
    output_rows = []

    for _, cyc in df_cycles.iterrows():
        start, end = cyc["cycle_start"], cyc["cycle_end"]

        # Filter for rows in this sweep
        mask = (df[timestamp_col] >= start) & (df[timestamp_col] <= end)
        subset = df.loc[mask]

        if subset.empty:
            print(f"⚠ Warning: empty subset for sweep starting {start}. Skipping.")
            continue

        # Forward/backfill to get final voltage values
        valid_vals = subset[valid_volt_cols].ffill().bfill()
        if valid_vals.empty:
            print(f"⚠ Warning: no valid volt data for sweep starting {start}. Skipping.")
            continue

        volt_values = valid_vals.iloc[-1]  # final values

        # Build output row
        row_out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": cyc["duration_sec"],
            "interval_sec": cyc["interval_sec"]
        }

        for col in valid_volt_cols:
            row_out[col] = volt_values[col]

        output_rows.append(row_out)

    return pd.DataFrame(output_rows)


volt_cols = [c for c in df_raw.columns if c.startswith("pack_cellvoltage_")]

df_sweeps_572 = extract_voltage_sweeps(df_raw, volt_cols)
df_sweeps_572


In [ ]:
df_sweeps_572.to_csv("df_sweeps_572.csv", index=False)

In [ ]:
df_sweeps_572.interval_sec.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
def extract_temperature_sweeps(
    decoded,
    temp_cols,
    timestamp_col="timestamp_ist",
    ignore_sensors=None
):
    """
    Detects temperature sweeps (coverage cycles).
    Works with ANY column naming pattern:
        pack_temperature1
        pack_temperature_1
        Pack_Temperature_1
    as long as the column ends with digits.

    ignore_sensors: set of sensor numbers to exclude.
    """

    # -----------------------------------------
    # 1. Extract numeric sensor IDs robustly
    # -----------------------------------------
    def get_sensor_num(colname):
        # pull all digits from the column name
        digits = ''.join(ch for ch in colname if ch.isdigit())
        return int(digits) if digits else None

    if ignore_sensors is None:
        ignore_sensors = set()

    valid_temp_cols = []
    sensor_ids = []

    for c in temp_cols:
        sid = get_sensor_num(c)
        if sid is None:
            # skip weird columns
            continue
        if sid not in ignore_sensors:
            valid_temp_cols.append(c)
            sensor_ids.append(sid)

    EXPECTED = len(valid_temp_cols)

    if EXPECTED == 0:
        print("⚠ No valid temperature columns found after parsing sensor numbers.")
        return pd.DataFrame()

    print(f"Expecting {EXPECTED} temperature sensors per sweep.")

    # -----------------------------------------
    # 2. Sort + compute sensors_present
    # -----------------------------------------
    decoded = decoded.sort_values(timestamp_col).reset_index(drop=True)

    decoded["sensors_present"] = decoded[valid_temp_cols].notna().apply(
        lambda row: [sensor_ids[i] for i, ok in enumerate(row) if ok],
        axis=1
    )

    # -----------------------------------------
    # 3. Detect sweeps
    # -----------------------------------------
    sweeps = []
    seen = set()
    cycle_start = None

    for _, row in decoded.iterrows():
        ts = row[timestamp_col]
        sensors = row["sensors_present"]

        if len(seen) == 0:
            cycle_start = ts

        seen.update(sensors)

        if len(seen) == EXPECTED:
            sweeps.append({"cycle_start": cycle_start, "cycle_end": ts})
            seen = set()

    # -----------------------------------------
    # 4. No sweeps found
    # -----------------------------------------
    if not sweeps:
        print("⚠ No complete temperature sweeps detected.")
        cols = ["cycle_start", "cycle_end", "duration_sec", "interval_sec"] + valid_temp_cols
        return pd.DataFrame(columns=cols)

    df_cycles = pd.DataFrame(sweeps)
    df_cycles["duration_sec"] = (
        df_cycles["cycle_end"] - df_cycles["cycle_start"]
    ).dt.total_seconds()

    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None

    # -----------------------------------------
    # 5. Extract final temperature values
    # -----------------------------------------
    rows_out = []

    for _, cyc in df_cycles.iterrows():
        start = cyc["cycle_start"]
        end = cyc["cycle_end"]

        mask = (decoded[timestamp_col] >= start) & (decoded[timestamp_col] <= end)
        segment = decoded.loc[mask]

        if segment.empty:
            continue

        vals = segment[valid_temp_cols].ffill().bfill()
        if vals.empty:
            continue

        final_vals = vals.iloc[-1]

        out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": cyc["duration_sec"],
            "interval_sec": cyc["interval_sec"],
        }

        for col in valid_temp_cols:
            out[col] = final_vals[col]

        rows_out.append(out)

    return pd.DataFrame(rows_out)



temp_cols = [c for c in df_raw.columns if c.startswith("pack_temperature")]
# For example: If sensors 7, 51 are permanently dead
# ignore = {81, 82, 83, 84}  # adjust as needed
ignore_dead = [81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108]   # or {7, 51} if some sensors never show up

df_temp_sweeps = extract_temperature_sweeps(
    df_raw,
    temp_cols,
    timestamp_col="timestamp_ist",
    ignore_sensors=ignore_dead
)

df_temp_sweeps.head()

## CSV to decoded output

In [ ]:
import pandas as pd

# Load your file
df = pd.read_csv("bb_id28_6084_271125_1300hrs_10mins.csv")
display(df.head())
# If your columns have duplicate names like timestamp,timestamp,
# pandas will automatically rename them timestamp and timestamp.1
# Let's identify the actual name of the second timestamp column:
ts_cols = [c for c in df.columns if c.startswith("timestamp")]

# Keep only the FIRST timestamp column
timestamp_col = ts_cols[0]

# Build the desired column order
byte_cols = [f"byte{i}" for i in range(1, 9)]

new_order = [
    "id",
    timestamp_col,
    "sequence",
    "can_id",
] + byte_cols

# Reorder the dataframe
df_reordered = df[new_order]

# Save back to CSV
df_reordered.to_csv("bb_id28_6084_271125_1300hrs_10mins_reordered.csv", index=False)
df_reordered.head()


In [ ]:
df_reordered.head().to_csv("df_reordered_head.csv", index=False)

In [ ]:
# df_reordered["can_id_adj"] = drop_msb_and_convert(df_reordered["can_id"].tolist())

In [ ]:
def drop_msb(can_id):
    """
    Removes the 31st bit from the DBC CAN ID.
    Produces the raw CAN ID observed in your logs.
    """
    return can_id & 0x7FFFFFFF


In [ ]:
dbc_lines = open("canId_conversions.txt").read().splitlines()

dbc_ids = []

for line in dbc_lines:
    if line.startswith("BO_ "):
        parts = line.split()
        dbc_id = int(parts[1])
        dbc_ids.append(dbc_id)

dbc_ids = sorted(set(dbc_ids))
len(dbc_ids)
raw_ids = [drop_msb(cid) for cid in dbc_ids]


In [ ]:
raw_ids = [drop_msb(cid) for cid in dbc_ids]

df_map = pd.DataFrame({
    "dbc_can_id": dbc_ids,
    "raw_can_id": raw_ids,
})

df_map["dbc_hex"] = df_map["dbc_can_id"].apply(lambda x: hex(x))
df_map["raw_hex"] = df_map["raw_can_id"].apply(lambda x: hex(x))

df_map.to_csv("dbc_to_raw_can_id_mapping.csv", index=False)


In [ ]:
df_map = df_map.rename(columns={
    "dbc_can_id": "dbc_id",
    "raw_can_id": "raw_id"
})
df_map.head()

In [ ]:
# RAW → DBC (for decoding incoming CAN messages)
raw_to_dbc = dict(zip(df_map["raw_id"], df_map["dbc_id"]))

# DBC → RAW (if you ever need it)
dbc_to_raw = dict(zip(df_map["dbc_id"], df_map["raw_id"]))


In [ ]:
df_reordered["dbc_id"] = df_reordered["can_id"].map(raw_to_dbc)
df_reordered.dbc_id = df_reordered.dbc_id.astype("Int64")
df_reordered[~df_reordered["dbc_id"].isna()].head()

In [ ]:
df_reordered.dbc_id.sort_values().unique()

In [ ]:
decoded = df_reordered[["id","timestamp","sequence"]].copy()

# Empty columns
for i in range(1, 577):
    decoded[f"Pack_cellVoltage_{i}"] = None

for i in range(1, 109):
    decoded[f"Pack_Temperature_{i}"] = None

byte_cols = [f"byte{i}" for i in range(1,9)]
df_reordered["payload"] = df_reordered[byte_cols].values.tolist()
display(df_reordered.head())
display(decoded[decoded.Pack_cellVoltage_1.notnull()].head())

In [ ]:
for idx, row in df_reordered.iterrows():
    dbc = row["dbc_id"]
    bytes_ = row["payload"]
    
    # Temperature frames
    if dbc in temp_map:
        sensor_indices = temp_map[dbc]
        for k, sensor_id in enumerate(sensor_indices):
            if sensor_id <= 108:
                decoded.loc[idx, f"Pack_Temperature_{sensor_id}"] = bytes_[k] - 40

    # Voltage frames
    if dbc in volt_map:
        cells = volt_map[dbc]
        for k, cell_index in enumerate(cells):
            L = bytes_[k*2]       # lower byte
            H = bytes_[k*2 + 1]   # higher byte

            raw_val = (H << 8) | L
            decoded_value = raw_val * 0.001  # as per DBC scaling

            decoded.loc[idx, f"Pack_cellVoltage_{cell_index}"] = decoded_value


In [ ]:
decoded.sort_values(by="timestamp", inplace=True)

In [ ]:
# Compute the correct position
ts_index = decoded.columns.get_loc("timestamp")

# Insert at index+1
decoded.insert(loc=ts_index + 1, column="timestamp_ist",
          value=(pd.to_datetime(decoded["timestamp"], unit="ms", utc=True).dt.tz_convert("Asia/Kolkata").dt.tz_localize(None)))

# decoded["timestamp_ist"] = pd.to_datetime(decoded["timestamp_ist"], format="%Y-%m-%d %H:%M:%S.%f")

In [ ]:
# decoded[decoded.Pack_cellVoltage_1.notna()|decoded.Pack_cellVoltage_5.notna()|decoded.Pack_cellVoltage_9.notna()|decoded.Pack_cellVoltage_13.notna()|decoded.Pack_cellVoltage_17.notna()|decoded.Pack_cellVoltage_21.notna()|decoded.Pack_cellVoltage_25.notna()]

In [ ]:
# Get all cell voltage columns
volt_cols = [c for c in decoded.columns if c.startswith("Pack_cellVoltage_")]

# Extract numeric index from each column
cell_indices = [int(c.split("_")[-1]) for c in volt_cols]
num_cells = len(cell_indices)  # should be 576

decoded["cells_present"] = decoded[volt_cols].notna().apply(lambda row: 
    [cell_indices[i] for i, present in enumerate(row) if present], axis=1)

cycles = []
current_cells = set()
cycle_start_time = None

for idx, row in decoded.iterrows():
    cells = row["cells_present"]
    ts = row["timestamp_ist"]

    # Start of a new cycle
    if len(current_cells) == 0:
        cycle_start_time = ts

    # Add newly seen cells
    current_cells.update(cells)

    # If full cycle achieved
    if len(current_cells) == num_cells:
        cycle_end_time = ts
        duration = (cycle_end_time - cycle_start_time).total_seconds()

        cycles.append({
            "cycle_start": cycle_start_time,
            "cycle_end": cycle_end_time,
            "duration_sec": duration
        })

        # Reset for next cycle
        current_cells = set()

df_cycles = pd.DataFrame(cycles)
display(df_cycles)


In [ ]:
# volt_cols = [c for c in decoded.columns if c.startswith("Pack_cellVoltage_")]
# cell_indices = [int(c.split("_")[-1]) for c in volt_cols]

def extract_cycle_cell_values(decoded, volt_cols):
    """
    Detects full 576-cell cycles and extracts:
      - cycle_start
      - cycle_end
      - duration_sec
      - interval_sec
      - all 576 Pack_cellVoltage columns

    Returns:
        DataFrame with one row per complete cycle
    """

    cell_indices = [int(c.split("_")[-1]) for c in volt_cols]
    num_cells = len(cell_indices)

    cycles = []
    current_cells = set()
    cycle_start_time = None

    # STEP 1 — DETECT CYCLES
    for idx, row in decoded.iterrows():
        cells_present = [
            cell_indices[i] for i, present in enumerate(row[volt_cols].notna()) if present
        ]
        ts = row["timestamp_ist"]

        # Start of new sweep
        if len(current_cells) == 0:
            cycle_start_time = ts

        # Add cells detected in this row
        current_cells.update(cells_present)

        # Full sweep detected
        if len(current_cells) == num_cells:
            cycle_end_time = ts

            cycles.append({
                "cycle_start": cycle_start_time,
                "cycle_end": cycle_end_time
            })

            # Reset for next cycle
            current_cells = set()

    # STEP 2 — CALCULATE DURATION + INTERVAL
    df_cycles = pd.DataFrame(cycles)
    df_cycles["duration_sec"] = (df_cycles["cycle_end"] - df_cycles["cycle_start"]).dt.total_seconds()
    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None  # first cycle has no previous interval

    # STEP 3 — EXTRACT 576 CELL VALUES FOR EACH CYCLE
    result_rows = []

    for i, row in df_cycles.iterrows():
        start, end = row["cycle_start"], row["cycle_end"]

        # Filter data inside this cycle interval
        mask = (decoded["timestamp_ist"] >= start) & (decoded["timestamp_ist"] <= end)
        segment = decoded.loc[mask, volt_cols]

        # Forward/backfill and take the final available value
        final_values = segment.ffill().bfill().iloc[-1]

        # Build output row
        out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": row["duration_sec"],
            "interval_sec": row["interval_sec"],
        }

        # Append all 576 cell values
        for col in volt_cols:
            out[col] = final_values[col]

        result_rows.append(out)

    return pd.DataFrame(result_rows)

volt_cols = [c for c in decoded.columns if c.startswith("Pack_cellVoltage_")]

df_cycle_voltages = extract_cycle_cell_values(decoded, volt_cols)

display(df_cycle_voltages)

In [ ]:
df_cycle_voltages.to_csv("df_cycle_voltages_rawcan_10mins.csv", index=False)  

In [ ]:
df_cycle_voltages.interval_sec.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
df_cycle_voltages.interval_sec.describe()

In [ ]:
df_cycle_voltages.cycle_start.min(),df_cycle_voltages.cycle_end.max()

In [ ]:
import pandas as pd

def extract_cycle_temperature_values(
    decoded,
    temp_cols,
    timestamp_col="timestamp_ist",
    ignore_sensors=None
):
    """
    Detects complete temperature cycles, mirroring extract_cycle_cell_values.

    A cycle ends when all expected temperature sensors (except ignored ones)
    have appeared at least once.

    Parameters
    ----------
    decoded : pd.DataFrame
        Must contain temperature columns and a timestamp column.
    temp_cols : list[str]
        List of Pack_Temperature_* columns.
    timestamp_col : str
        Timestamp column name.
    ignore_sensors : set[int]
        Sensor IDs to ignore entirely (dead sensors).

    Returns
    -------
    pd.DataFrame
        cycle_start, cycle_end, duration_sec, interval_sec
        + final temperature values for each sensor in that cycle.
    """

    # ------------------------------
    # STEP 1 — Determine valid sensors
    # ------------------------------
    def get_sensor_num(colname):
        return int(colname.split("_")[-1])

    if ignore_sensors is None:
        ignore_sensors = set()

    valid_temp_cols = [
        c for c in temp_cols
        if get_sensor_num(c) not in ignore_sensors
    ]

    sensor_ids = [get_sensor_num(c) for c in valid_temp_cols]
    EXPECTED = len(sensor_ids)

    if EXPECTED == 0:
        print("⚠ No valid temperature columns after exclusions.")
        return pd.DataFrame()

    print(f"Expecting {EXPECTED} temperature sensors per cycle.")

    # ------------------------------
    # STEP 2 — Sort + compute sensors_present
    # ------------------------------
    decoded = decoded.sort_values(timestamp_col).reset_index(drop=True)

    decoded["sensors_present"] = decoded[valid_temp_cols].notna().apply(
        lambda row: [sensor_ids[i] for i, ok in enumerate(row) if ok],
        axis=1
    )

    # ------------------------------
    # STEP 3 — Cycle detection
    # ------------------------------
    cycles = []
    seen = set()
    cycle_start = None

    for _, row in decoded.iterrows():
        ts = row[timestamp_col]
        sensors = row["sensors_present"]

        if len(seen) == 0:
            cycle_start = ts

        seen.update(sensors)

        if len(seen) == EXPECTED:
            cycles.append({"cycle_start": cycle_start, "cycle_end": ts})
            seen = set()

    # ------------------------------
    # STEP 4 — No cycles found
    # ------------------------------
    if not cycles:
        print("⚠ No complete temperature cycles detected.")
        cols = ["cycle_start", "cycle_end", "duration_sec", "interval_sec"] + valid_temp_cols
        return pd.DataFrame(columns=cols)

    df_cycles = pd.DataFrame(cycles)
    df_cycles["duration_sec"] = (
        df_cycles["cycle_end"] - df_cycles["cycle_start"]
    ).dt.total_seconds()
    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None

    # ------------------------------
    # STEP 5 — Extract final values for each cycle
    # ------------------------------
    out_rows = []

    for _, cyc in df_cycles.iterrows():
        start = cyc["cycle_start"]
        end = cyc["cycle_end"]

        mask = (decoded[timestamp_col] >= start) & (decoded[timestamp_col] <= end)
        subset = decoded.loc[mask]

        if subset.empty:
            continue

        vals = subset[valid_temp_cols].ffill().bfill()
        if vals.empty:
            continue

        final_vals = vals.iloc[-1]

        row_out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": cyc["duration_sec"],
            "interval_sec": cyc["interval_sec"],
        }

        for col in valid_temp_cols:
            row_out[col] = final_vals[col]

        out_rows.append(row_out)

    return pd.DataFrame(out_rows)


temp_cols = [
    c for c in decoded.columns
    if c.startswith("Pack_Temperature_")
]

ignore_dead = set()   # or {7, 51} if some sensors never show up
ignore_dead = [81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108]   # or {7, 51} if some sensors never show up

df_temp_cycles = extract_cycle_temperature_values(
    decoded,
    temp_cols,
    timestamp_col="timestamp_ist",
    ignore_sensors=ignore_dead
)

df_temp_cycles.head()
